In [ ]:
import requests
import pandas as pd


ivy_league_schools = {
    "Harvard University": {"city": "Cambridge", "state": "MA"},
    "Yale University": {"city": "New Haven", "state": "CT"},
    "Princeton University": {"city": "Princeton", "state": "NJ"},
    "Columbia University": {"city": "New York", "state": "NY"},
    "University of Pennsylvania": {"city": "Philadelphia", "state": "PA"},
    "Brown University": {"city": "Providence", "state": "RI"},
    "Dartmouth College": {"city": "Hanover", "state": "NH"},
    "Cornell University": {"city": "Ithaca", "state": "NY"}
}

base_url_schools = "https://api.data.gov/ed/collegescorecard/v1/schools"
all_schools_data = []

for school_name, location in ivy_league_schools.items():
    params = {
        "school.name": school_name,
        "fields": "id,school.name,school.city,school.state,location.lat,location.lon",
        "api_key": "aUYfWIfRdwDrbSfTIm3lTm5M67bWMk1YQNaqz4go"
    }
    
    response = requests.get(base_url_schools, params=params)
    data = response.json()
    
    if data.get('results'):
        for result in data['results']:
            if result.get('school.name') == school_name:
                all_schools_data.append(result)
                

# Creates the data frame for the schools
schools_df = pd.DataFrame(all_schools_data)

# This is where the APi data is stored for the weather. 
base_url_weather = "https://archive-api.open-meteo.com/v1/archive"

weather_summary = []

for index, school in schools_df.iterrows():
    lat = school['location.lat']
    lon = school['location.lon']
    school_name = school['school.name']
    city = school['school.city']
    state = school['school.state']
    
    params = {
        "latitude": lat,
        "longitude": lon,
        "daily": ["temperature_2m_min", "sunshine_duration"],
        "timezone": "America/New_York",
        "temperature_unit": "fahrenheit",
        "start_date": "2026-01-01",
        "end_date": "2026-01-31"
    }
    
    response = requests.get(base_url_weather, params=params)
    weather_data = response.json()
    
    if 'daily' in weather_data:
        # Convert sunshine from seconds to hours
        temps = weather_data['daily']['temperature_2m_min']
        sunshine_seconds = weather_data['daily']['sunshine_duration']
        sunshine_hours = [s / 3600 if s is not None else 0 for s in sunshine_seconds]
        
        # Count freezing days (< 32°F)
        freezing_days = sum(1 for temp in temps if temp is not None and temp < 32)
        
        # Count low sunshine days (< 5 hours); a rule I decided since it's low visibility and cloudly. 
        low_sunshine_days = sum(1 for hours in sunshine_hours if hours < 5)
        #this modifies the data frame for each school, location and my metrics for bad weather.
        weather_summary.append({
            'school_name': school_name,
            'city': city,
            'state': state,
            'latitude': lat,
            'longitude': lon,
            'days_below_32F': freezing_days,
            'days_under_5hrs_sun': low_sunshine_days
        })
        
       
# Create final summary DataFrame
summary_df = pd.DataFrame(weather_summary)

# Display results
print("\n" + "="*80)
print("IVY LEAGUE SCHOOLS - JANUARY 2026 WEATHER SUMMARY")
print("="*80)
print(summary_df.to_string(index=False))


# Optional: Sort by worst weather
print("\n" + "="*80)
print("SCHOOLS RANKED BY FREEZING DAYS")
print("="*80)
print(summary_df.sort_values('days_below_32F', ascending=False).to_string(index=False))